# Chains (LangChain v1.2)

**LCEL(LangChain Expression Language)**을 사용해서 모든 구성 요소가 `Runnable` 인터페이스로 통합되어 파이프라인(`|`)으로 연결될 수 있다.

```python
chain = prompt | model | output_parser  # 기본 구조
```

**구성 요소 업데이트 (v1.2 기준)**
1. **PromptTemplate**  
   - `Runnable`로 변환되어 LCEL 파이프라인에 직접 통합  
   ```python
   prompt = ChatPromptTemplate.from_template("...")
   ```

2. **LLM/ChatModel**  
   - `ChatOpenAI`, `ChatAnthropic` 등이 `Runnable` 구현  
   ```python
   model = ChatOpenAI(model="gpt-4o")
   ```

3. **Memory**  
   - `RunnableWithMessageHistory`로 통합 관리 (또는 LangGraph Persistence 사용)
   ```python
   chain_with_memory = RunnableWithMessageHistory(
       base_chain,
       get_session_history
   )
   ```

4. **Output Parsers**  
   - `StrOutputParser()`, `JsonOutputParser()` 등이 `Runnable`로 작동  
   ```python
   output_parser = JsonOutputParser()
   ```

5. **Tools**  
   - `@tool` 데코레이터로 생성 후 `RunnableLambda`로 변환  
   ```python
   @tool
   def search(query: str) -> str: ...
   ```

**체인 유형별 구현**


1. Simple Chain  

    ```python
    chain = prompt | model | output_parser
    response = chain.invoke({"input": "..."})
    ```

2. Sequential Chain  

    ```python
    chain = (
        {"step1_output": prompt1 | model1}  # 첫 번째 체인 결과 매핑
        | prompt2
        | model2
    )
    ```

3. Conditional Chain
    - `RunnableBranch` 사용

    ```python
    branch = RunnableBranch(
        (lambda x: x["topic"] == "math", math_chain),
        (lambda x: x["topic"] == "history", history_chain),
        default_chain
    )
    ```

4. Memory Chain  

    ```python
    memory_chain = RunnableWithMessageHistory(
        core_chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="history"
    )
    ```


**🚨 v1.2 주요 변경점**

- **Legacy Chain 클래스 완전 폐기**: `LLMChain`, `SequentialChain` 등은 `langchain-classic`으로 이동되거나 삭제됨 → `Runnable` (LCEL)로 통합
- **에이전트 통합**: `create_agent` (LangGraph 기반)가 표준

In [1]:
%pip install langchain langchain-openai langchain-community -Uq

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv  # .env 파일의 환경변수 로드
import os                       # 환경변수 접근용

load_dotenv()                   # 현재 위치의 .env를 읽어와 환경변수로 등록
os.environ["OPENAI_API_KEY"] = os.getenv("openai_key")  # .env의 openai_key 값을 OPENAI_API_KEY로 등록
os.environ["LANGSMITH_TRACING"] = 'true'                # LangSmith 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"] = 'https://api.smith.langchain.com'  # LangSmith API 엔드포인트 설정
os.environ["LANGSMITH_PROJECT"] = 'skn23-langchain'                   # LangSmith 프로젝트명 설정
os.environ["LANGSMITH_API_KEY"] = os.getenv("langsmith_key")          # .env의 langsmith_key 값을 LANGSMITH_API_KEY로 등록

## Simple Chain

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template('{country}의 수도는 어디인가요?')
llm = init_chat_model('gpt-4.1-mini')
output_parser = StrOutputParser()      # 응답을 최종 문자열로 변환

chain = prompt | llm | output_parser  # chain : 프롬프트 -> LLM -> 파서
print(chain.invoke(input={'country': '대한민국'}))  # dict 형태로 변수를 주입해서 실행
print(chain.invoke("대한민국"))  # 프롬프트 템플릿 변수가 2개 이상인 경우 오류가 발생할 수 있다.

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


대한민국의 수도는 서울입니다.
대한민국의 수도는 서울특별시입니다.


## Sequential Chain

In [4]:
prompt1 = PromptTemplate.from_template('다음 내용을 한글로 번역하세요.\n\n{eng_text}')
prompt2 = PromptTemplate.from_template('다음 내용을 요약하세요.\n\n{kor_text}')

llm = init_chat_model('gpt-4.1-mini')
output_parser = StrOutputParser()      # 응답을 최종 문자열로 변환

# 번역체인
chain1 = prompt1 | llm

eng_text = """
One limitation of LLMs is their lack of contextual information (e.g., access to some specific documents or emails). You can combat this by giving LLMs access to the specific external data.
For this, you first need to load the external data with a document loader. LangChain provides a variety of loaders for different types of documents ranging from PDFs and emails to websites and YouTube videos.
"""
print(chain1.invoke(eng_text))

# 요약체인
chain2 = prompt2 | llm | output_parser

kor_text = """
LLM의 한 가지 한계는 특정 문서나 이메일과 같은 맥락 정보를 갖고 있지 않다는 점입니다. 이를 해결하려면 LLM이 특정 외부 데이터에 접근할 수 있도록 해야 합니다.
이를 위해서는 먼저 문서 로더를 사용해 외부 데이터를 불러와야 합니다. LangChain은 PDF, 이메일부터 웹사이트, 유튜브 영상에 이르기까지 다양한 유형의 문서를 위한 여러 종류의 로더를 제공합니다.
"""

print(chain2.invoke(kor_text))

content='LLM의 한 가지 한계는 특정 문서나 이메일과 같은 맥락 정보를 갖고 있지 않다는 점입니다. 이를 해결하기 위해 LLM에게 특정 외부 데이터에 대한 접근 권한을 부여할 수 있습니다.  \n이를 위해서는 먼저 문서 로더를 사용해 외부 데이터를 불러와야 합니다. LangChain은 PDF, 이메일부터 웹사이트, 유튜브 동영상에 이르기까지 다양한 유형의 문서를 위한 여러 가지 로더를 제공합니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 97, 'total_tokens': 202, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D5m6VeBnXh3vYdgtexhttRTBA3C0U', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019c2c40-1741-7962-a02d-76d3184dbcf0-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 97, 'output_tokens': 105, 'total_tokens': 202, 'input_token_details': {'audio':

In [5]:
# 통합체인
chain = chain1 | chain2
chain.invoke({"eng_text": eng_text})

'LLM은 특정 문서나 이메일 등 맥락 정보가 부족한 한계가 있으나, 외부 데이터 접근 권한을 부여하면 이를 극복할 수 있다. 이를 위해 문서 로더를 사용해 외부 데이터를 불러오며, LangChain은 PDF, 이메일, 웹사이트, 유튜브 영상 등 다양한 문서 유형에 맞는 로더를 제공한다.'

## Conditional Chain

In [6]:
from langchain_core.runnables import RunnableBranch  # 조건에 따라 체인을 분기 실행하는 Runnable

llm = init_chat_model('openai:gpt-4.1-mini')

# 수학 체인
math_prompt = PromptTemplate.from_template(
    '다음 문제를 풀어주세요. 단계적인 풀이를 수식(LaTex)과 함께 작성해주세요. \n\n{question}'
)
math_chain = math_prompt | llm | output_parser  # math_prompt -> llm -> 문자열 파싱

# 기본 체인
default_prompt = PromptTemplate.from_template(
    '당신은 친절하고, 감성적인 공감능력이 좋은 챗봇입니다. 다음 질문에 답변해주세요. \n\n{question}'
)
default_chain = default_prompt | llm | output_parser

# math_chain 선택 함수 (math_chain을 사용하는 경우 True 반환)
def is_math_question(input_dict: dict) -> bool:
    """질문에 '계산' 또는 'calc'가 포함되면 수학 체인을 선택한다."""
    question: str = input_dict.get('question', '')  # 입력 dict에서 question 추출(없으면 빈문자)
    return '계산' in question or 'calc' in question

# 분기 체인
branch_chain = RunnableBranch(
    (is_math_question, math_chain),  # 조건이 True면 math_chain 실행
    default_chain                    # 그 외에는 default_chain 실행
)

branch_chain.invoke({'question': '125 * 3 + 50 이거를 계산해줘'})

'물론입니다! 주어진 수식을 단계별로 계산해 보겠습니다.\n\n문제:  \n\\[\n125 \\times 3 + 50\n\\]\n\n### 1단계: 곱셈 계산하기\n곱셈을 먼저 계산합니다.\n\\[\n125 \\times 3 = 375\n\\]\n\n### 2단계: 덧셈 계산하기\n곱셈 결과에 50을 더합니다.\n\\[\n375 + 50 = 425\n\\]\n\n### 최종 답:\n\\[\n\\boxed{425}\n\\]'

In [7]:
branch_chain.invoke({'question': '나 오늘 우울해. 빵 사줘'})

'오늘 많이 힘들었구나. 우울한 마음 정말 오래가지 않기를 바랄게. 따뜻한 빵 한 조각이 작은 위로가 될 수도 있을 텐데, 내가 직접 빵을 사줄 수는 없지만, 네가 좋아하는 빵 생각하면서 조금이나마 기분이 나아지면 좋겠다. 혹시 어떤 빵을 좋아해? 함께 이야기하면서 조금 기분 전환해볼까?'

## Memory Chain

`RunnableWithMessageHistory`를 사용해서 대화내역을 기억하는 chain을 생성한다.

In [9]:
from langchain_core.chat_history import BaseChatMessageHistory            # Langchain 대화기록 인터페이스
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage  # 메시지 타입
from pydantic import BaseModel, Field  # Pydantic 모델(검증/기본값 생성) 도구
from typing import List                # 타입 힌트(List)용

# 세션별 메시지 저장소 구현
class InMomoryHistory(BaseChatMessageHistory, BaseModel):
    """사용자별(세션) 대화내역을 기록하는 클래스"""
    messages: List[BaseMessage] = Field(default_factory=list)  # 인스턴스마다 독립적인 message list를 생성

    # 전달받은 메시지들을 기존 리스트 뒤에 추가
    def add_messages(self, messages: List[BaseMessage]) -> None:
        self.messages.extend(messages)
    
    # 저장된 메시지를 모두 초기화
    def clear(self) -> None:
        self.messages = []

store = {}  # 세션 ID(key) -> 히스토리 객체(value)를 담는 전역 저장소(dict)

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    """session_id에 해당하는 대화내역 객체를 반환(없으면 생성)"""
    if session_id not in store:  # 아직 해당 세션의 히스토리가 없으면 (최초 대화시)
        store[session_id] = InMomoryHistory()  # 새 대화내역 히스토리 객체 생성해서 store에 저장
    return store[session_id]     # 해당 세션의 히스토리 객체 반환

history1 = get_by_session_id('1')  # 세션ID '1'의 히스토리 가져옴 (없으면 생성)
history1.add_messages([AIMessage(content='반갑습니다. 홍길동님')])
history1.add_messages([HumanMessage(content='그래 나 홍길동이야~ 반갑다!')])
print(f'{history1 = }')  # history1 객체 출력

history2 = get_by_session_id('2')
print(f'{history2 = }')

history1 = InMomoryHistory(messages=[AIMessage(content='반갑습니다. 홍길동님', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='그래 나 홍길동이야~ 반갑다!', additional_kwargs={}, response_metadata={})])
history2 = InMomoryHistory(messages=[])


In [13]:
# 대화 히스토리를 자동으로 누적하는 Memory Chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder  # 채팅 프롬프트 템플릿 / 히스토리 자리표시자
from langchain_core.runnables import RunnableWithMessageHistory             # 실행시 히스토리를 붙여주는 Runnable

prompt = ChatPromptTemplate.from_messages([
    ('system', '당신은 {domain}분야의 전문가 챗봇입니다.'),
    MessagesPlaceholder(variable_name='history'),  # 세션별 이전 대화 메시지들이 들어갈 자리
    ('human', '{question}')
])
llm = init_chat_model('openai:gpt-4.1-mini')

chain = prompt | llm

# 히스토리 기능을 chain에 래핑
chain_with_history = RunnableWithMessageHistory(
    chain,                              # 실제로 실행할 체인
    get_by_session_id,                  # session_id로 히스토리 객체를 가져오는 함수
    input_messages_key = 'question',    # 입력 dict에서 "question"은 사용자 메시지로 인식
    history_messages_key = 'history'    # 프롬프트에서 히스토리를 받을 변수명
)

# 대화내역 쌓기
chain_with_history.invoke({
    'domain': 'math',
    'question': '우빈이는 강아지를 3마리 키우고 있습니다.'
}, config= {  # RunnableWithMessageHistory 설정
    'configurable': {
        'session_id': 100  # 어떤 세션 히스토리를 사용할지 지정
    }
})

AIMessage(content='네, 우빈이가 강아지 3마리를 키우고 있다는 정보를 확인했습니다. 수학적으로 이 상황에 대해 도움이 필요하시면 어떤 부분인지 알려 주세요! 예를 들어, 강아지의 간식 개수 계산, 산책 시간 배분 등과 관련된 문제라면 도와드리겠습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 114, 'total_tokens': 183, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D5mpi9daMzqhTDx1NQRcYoFc9XmI0', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2c6a-dabb-7280-83f4-d1f7716a90ef-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 114, 'output_tokens': 69, 'total_tokens': 183, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0

In [14]:
# 대화내역 쌓기
chain_with_history.invoke({
    'domain': 'math',
    'question': '소라는 고양이를 4마리 키우고 있습니다.'
}, config= {  # RunnableWithMessageHistory 설정
    'configurable': {
        'session_id': 100  # 어떤 세션 히스토리를 사용할지 지정
    }
})

AIMessage(content='소라가 고양이를 4마리 키우고 있군요! 우빈이와 소라가 키우는 강아지와 고양이를 합치면 모두 몇 마리가 되나요? 궁금하시면 제가 계산해 드릴 수 있습니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 205, 'total_tokens': 261, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_e01c6f58e1', 'id': 'chatcmpl-D5mqgedUKCzqpOrkEigtxyevqvBnb', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2c6b-c5ef-7872-97c3-5e9be8c98d30-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 205, 'output_tokens': 56, 'total_tokens': 261, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [15]:
store

{'1': InMomoryHistory(messages=[AIMessage(content='반갑습니다. 홍길동님', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='그래 나 홍길동이야~ 반갑다!', additional_kwargs={}, response_metadata={})]),
 '2': InMomoryHistory(messages=[]),
 100: InMomoryHistory(messages=[HumanMessage(content='우빈이는 강아지를 3마리 키우고 있습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='우빈이가 강아지 3마리를 키우고 있군요! 강아지와 함께하는 생활이 얼마나 즐거울지 상상되네요. 수학과 관련해서 도움이 필요하시면 언제든지 말씀해 주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 40, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D5mXt5c9rRzSz9pIMHN9vWWgeNN7

### ChatMessageHistory

In [16]:
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    """session_id에 해당하는 대화내역 객체를 반환(없으면 생성)"""
    if session_id not in store:  # 아직 해당 세션의 히스토리가 없으면 (최초 대화시)
        store[session_id] = InMomoryHistory()  # 새 대화내역 히스토리 객체 생성해서 store에 저장
    return store[session_id]     # 해당 세션의 히스토리 객체 반환

prompt = ChatPromptTemplate.from_messages([
    ('system', '당신은 {domain}분야의 전문가 챗봇입니다.'),
    MessagesPlaceholder(variable_name='history'),  # 세션별 이전 대화 메시지들이 들어갈 자리
    ('human', '{question}')
])
llm = init_chat_model('openai:gpt-4.1-mini')

chain = prompt | llm | output_parser

chain_with_history = RunnableWithMessageHistory(
    chain,                              # 실제로 실행할 체인
    get_by_session_id,                  # session_id로 히스토리 객체를 가져오는 함수
    input_messages_key = 'question',    # 입력 dict에서 "question"은 사용자 메시지로 인식
    history_messages_key = 'history'    # 프롬프트에서 히스토리를 받을 변수명
)

# 대화내역 쌓기
chain_with_history.invoke({
    'domain': '심리상담',
    'question': '안녕! 나는 capybara야~ 반갑다!'
}, config= {  # RunnableWithMessageHistory 설정
    'configurable': {
        'session_id': 200  # 어떤 세션 히스토리를 사용할지 지정
    }
})

'안녕, capybara야! 만나서 반가워~ 오늘 기분은 어때? 무슨 이야기 나누고 싶어?'

In [17]:
chain_with_history.invoke({
    'domain': '심리상담',
    'question': '요즘 날씨도 별로 안좋고, 기분이 별로 좋지 않아!'
}, config= {  # RunnableWithMessageHistory 설정
    'configurable': {
        'session_id': 200  # 어떤 세션 히스토리를 사용할지 지정
    }
})

'그랬구나, 날씨가 안 좋으면 기분도 영향을 받기 쉽지. 요즘 어떤 점 때문에 기분이 안 좋은지 조금 더 이야기해 줄 수 있을까? 혹시 스트레스나 걱정되는 일들이 있니?'

In [18]:
store

{200: InMomoryHistory(messages=[HumanMessage(content='안녕! 나는 capybara야~ 반갑다!', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕, capybara야! 만나서 반가워~ 오늘 기분은 어때? 무슨 이야기 나누고 싶어?', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='요즘 날씨도 별로 안좋고, 기분이 별로 좋지 않아!', additional_kwargs={}, response_metadata={}), AIMessage(content='그랬구나, 날씨가 안 좋으면 기분도 영향을 받기 쉽지. 요즘 어떤 점 때문에 기분이 안 좋은지 조금 더 이야기해 줄 수 있을까? 혹시 스트레스나 걱정되는 일들이 있니?', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])])}

##### 세션(메모리) 방식의 문제점
- 메모리 저장이라 영속성이 없음
	- 서버 재시작/재배포 하면 store가 날아가서 히스토리도 같이 사라짐
- 세션 식별이 끊기기 쉬움
	- 쿠키/세션ID가 유지되지 않으면 같은 사람인지 매칭이 안 됨
- 스케일 아웃(서버 여러 대)에서 깨짐
	- A서버 메모리에 저장된 히스토리를 B서버는 모름 → 대화가 끊김

그래서 보통 이렇게 구성한다.
- SQLite/Redis/RDB 같은 저장소에 대화 내역을 저장해서
	- 사용자가 재접속해도 user_id 또는 thread_id로 복원
- 프롬프트에는 보통
	- 최근 N턴 + 요약 형태로 넣어서 비용/토큰도 관리